In [2]:
!pip install pyspark

  Using cached pyspark-3.4.0.tar.gz (310.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
  Running setup.py install for pyspark: started
  Running setup.py install for pyspark: finished with status 'done'


You should consider upgrading via the 'C:\Users\Onur\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [73]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType, StringType

spark=SparkSession.builder.appName('FordGoBike').getOrCreate()
spark
# https://www.kaggle.com/code/priyankabnl/fordgobike-trip-data

In [130]:
path="G:/My Drive/Develhope/develhope-Data5-Team3/Week_1/2017-fordgobike-tripdataa.csv"
df_pyspark=spark.read.csv(path,header=True, inferSchema=True)

In [7]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [8]:
df_pyspark.head(5)


[Row(start_time='57:39.7', start time hour=4, start time minute=57, start time seconds=40, _c4='PM', end_time='12:50.2', end_time hour=3, end_time minute=12, end_time seconds=50, _c9='PM', start_station_id=74, start_station_name='Laguna St at Hayes St', start_station_latitude=37.77643482, start_station_longitude=-122.426244, end_station_id=43, end_station_name='San Francisco Public Library (Grove St at Hyde St)', end_station_latitude=37.7787677, end_station_longitude=-122.4159292, bike_id=96, user_type='Customer', member_birth_year=1987, member_gender='Male', pyment='credit card'),
 Row(start_time='56:34.8', start time hour=3, start time minute=56, start time seconds=35, _c4='PM', end_time='49:55.6', end_time hour=1, end_time minute=49, end_time seconds=56, _c9='PM', start_station_id=284, start_station_name='Yerba Buena Center for the Arts (Howard St at 3rd St)', start_station_latitude=37.78487208, start_station_longitude=-122.4008757, end_station_id=96, end_station_name='Dolores St at

In [131]:
df_pyspark=df_pyspark.withColumn('start_station_longitude',F.col('start_station_longitude').cast(FloatType()))\
    .withColumn('start_station_latitude',F.col('start_station_latitude').cast(FloatType()))\
    .withColumn('end_station_latitude',F.col('end_station_latitude').cast(FloatType()))\
    .withColumn('end_station_longitude',F.col('end_station_longitude').cast(FloatType()))
df_pyspark=df_pyspark.withColumnRenamed('_c4','start_am_pm').withColumnRenamed('_c9','end_am_pm')
df_pyspark.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- start time hour: integer (nullable = true)
 |-- start time minute: integer (nullable = true)
 |-- start time seconds: integer (nullable = true)
 |-- start_am_pm: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- end_time hour: integer (nullable = true)
 |-- end_time minute: integer (nullable = true)
 |-- end_time seconds: integer (nullable = true)
 |-- end_am_pm: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: float (nullable = true)
 |-- start_station_longitude: float (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: float (nullable = true)
 |-- end_station_longitude: float (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |

In [37]:
df_pyspark.describe().show()


+-------+----------+-----------------+------------------+------------------+-----------+--------+-----------------+------------------+------------------+---------+-----------------+--------------------+----------------------+-----------------------+-----------------+--------------------+--------------------+---------------------+------------------+----------+------------------+-------------+-----------+
|summary|start_time|  start time hour| start time minute|start time seconds|start_am_pm|end_time|    end_time hour|   end_time minute|  end_time seconds|end_am_pm| start_station_id|  start_station_name|start_station_latitude|start_station_longitude|   end_station_id|    end_station_name|end_station_latitude|end_station_longitude|           bike_id| user_type| member_birth_year|member_gender|     pyment|
+-------+----------+-----------------+------------------+------------------+-----------+--------+-----------------+------------------+------------------+---------+-----------------+-----

In [132]:
df_pyspark= df_pyspark.withColumn('start time hour',F.when(df_pyspark['start_am_pm']=='PM',df_pyspark["start time hour"]+12).otherwise(df_pyspark["start time hour"]))
df_pyspark= df_pyspark.withColumn('start time hour',F.when(df_pyspark['start time hour']==24,12).otherwise(df_pyspark["start time hour"]))

In [133]:
df_pyspark= df_pyspark.withColumn('end_time hour',F.when(df_pyspark['end_am_pm']=='PM',df_pyspark["end_time hour"]+12).otherwise(df_pyspark["end_time hour"]))
df_pyspark= df_pyspark.withColumn('end_time hour',F.when(df_pyspark['end_time hour']==24,12).otherwise(df_pyspark["end_time hour"]))

In [ ]:
df_pyspark.filter(F.col('end_time hour')==24).show()

In [ ]:
#fixing 1-digit values 
df_pyspark=df_pyspark.withColumn('start time hour',F.lpad(F.col('start time hour'),2,'0'))\
    .withColumn('start time minute',F.lpad(F.col('start time minute'),2,'0'))\
    .withColumn('start time seconds',F.lpad(F.col('start time seconds'),2,'0'))\
    .withColumn('end_time hour',F.lpad(F.col('end_time hour'),2,'0'))\
    .withColumn('end_time minute',F.lpad(F.col('end_time minute'),2,'0'))\
    .withColumn('end_time seconds',F.lpad(F.col('end_time seconds'),2,'0'))

In [134]:
#creating start time and end time columns with timestamp for easy comparison
df_pyspark=df_pyspark.withColumn('start time',F.to_timestamp(F.concat_ws(':',F.lpad(F.col('start time hour'),2,'0'),F.lpad(F.col('start time minute'),2,'0'),F.lpad(F.col('start time seconds'),2,'0'))))
df_pyspark=df_pyspark.withColumn('end time',F.to_timestamp(F.concat_ws(':',F.lpad(F.col('end_time hour'),2,'0'),F.lpad(F.col('end_time minute'),2,'0'),F.lpad(F.col('end_time seconds'),2,'0'))))

In [ ]:
df_pyspark.show()

In [135]:
#swapping end time and start time values if start time > end time
df_pyspark=df_pyspark.withColumn('end time',F.when(df_pyspark['end time']<df_pyspark['start time'], df_pyspark['start time'])\
                                 .otherwise(df_pyspark['end time']))
df_pyspark=df_pyspark.withColumn('start time',F.when(df_pyspark['end time']==df_pyspark['start time'], 
                                                     F.to_timestamp(F.concat_ws(':','end_time hour','end_time minute','end_time seconds')))\
                                 .otherwise(df_pyspark['start time']))

In [137]:
# df_pyspark=df_pyspark.withColumn('start_time',F.concat_ws(':',F.lpad(F.hour(F.col('start time')),2,'0'),F.lapd(F.minute(F.col('start time')),2,'0'),F.lapd(F.second(F.col('start time')),2,'0')))#\
    #.withColumn('end_time',F.concat_ws(':',F.lpad(F.hour('end time'),2,'0'),F.lpad(F.minute('end time'),2,'0'),F.lpad(F.second('end time'),2,'0')))

AttributeError: module 'pyspark.sql.functions' has no attribute 'lapd'

In [138]:
df_pyspark=df_pyspark.withColumn('start_time',df_pyspark['start time']).withColumn('end_time',df_pyspark['end time'])

In [139]:
df_pyspark.columns

['start_time',
 'start time hour',
 'start time minute',
 'start time seconds',
 'start_am_pm',
 'end_time',
 'end_time hour',
 'end_time minute',
 'end_time seconds',
 'end_am_pm',
 'start_station_id',
 'start_station_name',
 'start_station_latitude',
 'start_station_longitude',
 'end_station_id',
 'end_station_name',
 'end_station_latitude',
 'end_station_longitude',
 'bike_id',
 'user_type',
 'member_birth_year',
 'member_gender',
 'pyment',
 'start time',
 'end time']

In [140]:
#Dropping 'start time' and 'end time' columns
df_pyspark=df_pyspark.drop(*['start time','end time','start time hour',
 'start time minute',
 'start time seconds',
 'end_time hour',
 'end_time minute',
 'end_time seconds',])
df_pyspark.show()

+-------------------+-----------+-------------------+---------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|         start_time|start_am_pm|           end_time|end_am_pm|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|
+-------------------+-----------+-------------------+---------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+
|2023-05-07 15:12:50|         PM|2023-05-07 16:57:40|       PM|              74|Laguna St at Haye...|             37.776436|   

In [18]:
!pip install haversine

  Using cached haversine-2.8.0-py2.py3-none-any.whl (7.7 kB)


You should consider upgrading via the 'C:\Users\Onur\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [141]:
from haversine import haversine

def haversine_f(lat1, lon1, lat2, lon2):
    return haversine( (lat1, lon1), (lat2, lon2),unit='m',normalize=True )

haversine_udf = F.udf(haversine_f)

In [142]:
#Calculate haversine distance(Onur)
df_pyspark=df_pyspark.withColumn('haversine_distance', 
                    haversine_udf(F.col('start_station_latitude'), F.col('start_station_longitude'), 
                                  F.col('end_station_latitude'), F.col('end_station_longitude'))
                    )

In [143]:
#Calculate haversine distance in meters(Uros)
df_pyspark=df_pyspark.withColumn('haversine_distance',
                                 haversine_udf('start_station_latitude', 
                                               'start_station_longitude', 
                                               'end_station_latitude', 
                                               'end_station_longitude'))

In [144]:
#assign timestamp to start_time and end_time
#Calculate 'Diff_in_seconds' 
#Calculate 'Diff_in_minutes' 
#Calculate 'Trip_cost' 
df_pyspark=df_pyspark.withColumn('start_time',F.to_timestamp('start_time','HH:mm:ss'))\
    .withColumn('end_time',F.to_timestamp('end_time','HH:mm:ss'))\
    .withColumn('Diff_in_seconds',F.col('end_time').cast('long')-F.col('start_time').cast('long'))\
    .withColumn('Diff_in_minutes',(F.col('Diff_in_seconds')/60))\
    .withColumn('Trip_cost',(F.col('Diff_in_minutes')*0.35))

df_pyspark.printSchema()


root
 |-- start_time: timestamp (nullable = true)
 |-- start_am_pm: string (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- end_am_pm: string (nullable = true)
 |-- start_station_id: integer (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: float (nullable = true)
 |-- start_station_longitude: float (nullable = true)
 |-- end_station_id: integer (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: float (nullable = true)
 |-- end_station_longitude: float (nullable = true)
 |-- bike_id: integer (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: integer (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)
 |-- haversine_distance: string (nullable = true)
 |-- Diff_in_seconds: long (nullable = true)
 |-- Diff_in_minutes: double (nullable = true)
 |-- Trip_cost: double (nullable = true)



In [22]:
df_pyspark.groupBy("bike_id")\
    .agg(F.sum("haversine_distance").alias("sum_distance")).sort(F.desc("sum_distance"))

DataFrame[bike_id: int, sum_distance: double]

In [145]:
df_pyspark.show()

+-------------------+-----------+-------------------+---------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+-------------+-----------+------------------+---------------+------------------+------------------+
|         start_time|start_am_pm|           end_time|end_am_pm|start_station_id|  start_station_name|start_station_latitude|start_station_longitude|end_station_id|    end_station_name|end_station_latitude|end_station_longitude|bike_id| user_type|member_birth_year|member_gender|     pyment|haversine_distance|Diff_in_seconds|   Diff_in_minutes|         Trip_cost|
+-------------------+-----------+-------------------+---------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+-------+----------+-----------------+--------

In [156]:
df_pyspark.write.option("header",True).mode('overwrite').csv('G:/My Drive/Develhope/mycsv')

Py4JJavaError: An error occurred while calling o1339.csv.
: ExitCodeException exitCode=1: ChangeFileModeByMask error (87): The parameter is incorrect.



	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1007)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(Raw

In [24]:
df_pyspark.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\Onur\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 683, in main
RuntimeError: Python in worker has different version 3.9 than that in driver 3.11, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.
